# SVM Review

## SVC 기본 구현

In [1]:
import numpy as np
import pandas as pd
from sklearn import svm, datasets
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [2]:
iris = datasets.load_iris()
X = iris.data[:,:4]
y = iris.target

In [3]:
clf = svm.SVC(kernel = 'linear')
clf.fit(X, y)
y_pred = clf.predict(X)
confusion_matrix(y, y_pred)

array([[50,  0,  0],
       [ 0, 49,  1],
       [ 0,  0, 50]], dtype=int64)

## Kernel 변경 및 Scaling에 따른 정확도 변경 검증

scikit-learn 데이터 중 유방암 관련 데이터를 바탕으로 분류를 진행해보자.

In [4]:
data = datasets.load_breast_cancer()
cancer_df = pd.DataFrame(data = data.data, columns = data.feature_names)
cancer_df['target'] = data.target

In [5]:
cancer_df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115,0
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637,0
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820,0
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400,0


해당 문제는 30개의 변수를 바탕으로 유방암 유무를 검진하는 데이터다. 

In [6]:
cancer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

모든 변수가  non-null하고 수치형 변수이다. 분류 문제를 풀기 위해 다양한 모델을 적용하려고 한다.

kernel 변경에 따른 정확도 검증을 실시하려고 한다.

In [7]:
# kernel : linear
X = cancer_df.iloc[:,:-1]
y = cancer_df.iloc[:,-1]

clf = svm.SVC(kernel = 'linear')
scores = cross_val_score(clf, X, y, cv = 5)
print(scores.mean())

0.9455364073901569


In [8]:
# kernel : 비선형(rbf)

clf = svm.SVC(kernel = 'rbf')
scores = cross_val_score(clf, X, y, cv = 5)
print(scores.mean())

0.9121720229777983


비선형보다는 선형에서 cv 정확도가 높게 나왔다. 이에 대한 이유는, 비선형에서 overfitting이 발생했을 것이라고 예측할 수 있다.
이제 그러면 로지스틱 회귀와 결과를 비교해보자.

In [9]:
clf = LogisticRegression()
scores = cross_val_score(clf, X, y, cv = 5)
print(scores.mean())

0.9385188635305075


C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\USER\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

로지스틱회귀는 converge가 실패하면서, svm이 더 좋은 성능을 내고 있다.

### 표준화 적용시

In [10]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)

clf = svm.SVC(kernel = 'linear')
scores = cross_val_score(clf, X_scaled, y, cv = 5)
print(scores.mean())

0.9701443875174661


표준화 작업을 통해서 더 좋은 정확도를 얻어낼 수 있다.
하지만 왜 표준화가 필요한 것일까?

일반적으로 로지스틱 회귀에 비해서, svm의 경우는 새로운 데이터가 들어왔을 때 이에 대해서 상대적으로 더 안정적인 분류가 가능하다.
여기에, 표준화를 적용한다면, 새로운 데이터에 대해 조금 더 flexible하게 대응이 가능하다. 이에 따라, 표준화의 적용이 더 좋은 결과를 가져온다.